In [ ]:
import os
import glob
import copy
import re
import ast
import pandas as pd
import numpy as np
import nibabel
import scipy
import scipy.io
from pprint import pprint

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import _cov, LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.decomposition import PCA, FastICA
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib qt
#matplotlib.use('Qt5Agg')

import mne
from mne.io import read_raw_fif, concatenate_raws
from mne.viz import plot_alignment, set_3d_title
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
from mne.decoding import SlidingEstimator, cross_val_multiscore, Scaler, Vectorizer
from mne.decoding import UnsupervisedSpatialFilter

from mpl_toolkits.mplot3d import Axes3D  # noqa

In [ ]:
# Set intial parameters

tmin = -0.1
tmax = 0.8
baseline = (-0.1,0)

downsample_freq = 200
powerline_freq = (50, 100, 150, 200)
low_pass_freq = 0.05
high_pass_freq = 330

# Savitzky-Golay filter
window_length = 25
polyorder = 4
# Butterworth filter
l_freq = 0.5
h_freq = 40
order = 2

# Number of runs
n_runs = 12

# Subject
subj = "Subj42"

# Tail specify the hyperparameter (e.g. ica, autoreject, smoothing, ...)
tail_fname = tail_fname = "_savgol-"+str(window_length)+"-"+str(polyorder)+"_downsample-"+str(downsample_freq)
raw_fname = "raw_tsss_trans.fif"
epoched_fname = "epoched"+tail_fname+".mat"
log_fname = "log"+tail_fname+".txt"
information_fname = "information.mat"

data_folder = "data"
subject_folder = "subject\\"+subj
meg_folder = "meg"
figure_folder = "fig"
information_folder = "information"

project_path = 'F:\\Data Fusion Project'

raw_path = os.path.join(project_path,subject_folder,meg_folder)
epoched_path = os.path.join(project_path,subject_folder,meg_folder)
log_path = os.path.join(project_path,subject_folder,meg_folder)
figure_path = os.path.join(project_path,subject_folder,meg_folder,figure_folder)
information_path = os.path.join(project_path,data_folder,information_folder)

# Events
event_sample_list = ['N1S1TFA1', 'N1S1TFA2', 'N1S2TFA1', 'N1S2TFA2',
                      'N1S3TFA1', 'N1S3TFA2', 'N1S4TFA1', 'N1S4TFA2',
                      'N2S1TFA1', 'N2S1TFA2', 'N2S2TFA1', 'N2S2TFA2',
                      'N2S3TFA1', 'N2S3TFA2', 'N2S4TFA1', 'N2S4TFA2',
                      'N3S1TFA1', 'N3S1TFA2', 'N3S2TFA1', 'N3S2TFA2',
                      'N3S3TFA1', 'N3S3TFA2', 'N3S4TFA1', 'N3S4TFA2',
                      'N4S1TFA1', 'N4S1TFA2', 'N4S2TFA1', 'N4S2TFA2',
                      'N4S3TFA1', 'N4S3TFA2', 'N4S4TFA1', 'N4S4TFA2']

event_sample_dict = {'N1S1TFA1':1, 'N1S1TFA2':2, 'N1S2TFA1':3, 'N1S2TFA2':4,
                     'N1S3TFA1':5, 'N1S3TFA2':6, 'N1S4TFA1':7, 'N1S4TFA2':8,
                     'N2S1TFA1':9, 'N2S1TFA2':10, 'N2S2TFA1':11, 'N2S2TFA2':12,
                     'N2S3TFA1':13, 'N2S3TFA2':14, 'N2S4TFA1':15, 'N2S4TFA2':16,
                     'N3S1TFA1':17, 'N3S1TFA2':18, 'N3S2TFA1':19, 'N3S2TFA2':20,
                     'N3S3TFA1':21, 'N3S3TFA2':22, 'N3S4TFA1':23, 'N3S4TFA2':24,
                     'N4S1TFA1':25, 'N4S1TFA2':26, 'N4S2TFA1':27, 'N4S2TFA2':28,
                     'N4S3TFA1':29, 'N4S3TFA2':30, 'N4S4TFA1':31, 'N4S4TFA2':32}

event_sample_dict_N = {'N1/N1S1TFA1':1, 'N1/N1S1TFA2':2, 'N1/N1S2TFA1':3, 'N1/N1S2TFA2':4,
                      'N1/N1S3TFA1':5, 'N1/N1S3TFA2':6, 'N1/N1S4TFA1':7, 'N1/N1S4TFA2':8,
                      'N2/N2S1TFA1':9, 'N2/N2S1TFA2':10, 'N2/N2S2TFA1':11, 'N2/N2S2TFA2':12,
                      'N2/N2S3TFA1':13, 'N2/N2S3TFA2':14, 'N2/N2S4TFA1':15, 'N2/N2S4TFA2':16,
                      'N3/N3S1TFA1':17, 'N3/N3S1TFA2':18, 'N3/N3S2TFA1':19, 'N3/N3S2TFA2':20,
                      'N3/N3S3TFA1':21, 'N3/N3S3TFA2':22, 'N3/N3S4TFA1':23, 'N3/N3S4TFA2':24,
                      'N4/N4S1TFA1':25, 'N4/N4S1TFA2':26, 'N4/N4S2TFA1':27, 'N4/N4S2TFA2':28,
                      'N4/N4S3TFA1':29, 'N4/N4S3TFA2':30, 'N4/N4S4TFA1':31, 'N4/N4S4TFA2':32}

event_sample_dict_TFA = {'TFA1/N1S1TFA1':1, 'TFA2/N1S1TFA2':2, 'TFA1/N1S2TFA1':3, 'TFA2/N1S2TFA2':4,
                        'TFA1/N1S3TFA1':5, 'TFA2/N1S3TFA2':6, 'TFA1/N1S4TFA1':7, 'TFA2/N1S4TFA2':8,
                        'TFA1/N2S1TFA1':9, 'TFA2/N2S1TFA2':10, 'TFA1/N2S2TFA1':11, 'TFA2/N2S2TFA2':12,
                        'TFA1/N2S3TFA1':13, 'TFA2/N2S3TFA2':14, 'TFA1/N2S4TFA1':15, 'TFA2/N2S4TFA2':16,
                        'TFA1/N3S1TFA1':17, 'TFA2/N3S1TFA2':18, 'TFA1/N3S2TFA1':19, 'TFA2/N3S2TFA2':20,
                        'TFA1/N3S3TFA1':21, 'TFA2/N3S3TFA2':22, 'TFA1/N3S4TFA1':23, 'TFA2/N3S4TFA2':24,
                        'TFA1/N4S1TFA1':25, 'TFA2/N4S1TFA2':26, 'TFA1/N4S2TFA1':27, 'TFA2/N4S2TFA2':28,
                        'TFA1/N4S3TFA1':29, 'TFA2/N4S3TFA2':30, 'TFA1/N4S4TFA1':31, 'TFA2/N4S4TFA2':32}

event_sample_dict_S = {'S1/N1S1TFA1':1, 'S1/N1S1TFA2':2, 'S2/N1S2TFA1':3, 'S2/N1S2TFA2':4,
                      'S3/N1S3TFA1':5, 'S3/N1S3TFA2':6, 'S4/N1S4TFA1':7, 'S4/N1S4TFA2':8,
                      'S1/N2S1TFA1':9, 'S1/N2S1TFA2':10, 'S2/N2S2TFA1':11, 'S2/N2S2TFA2':12,
                      'S3/N2S3TFA1':13, 'S3/N2S3TFA2':14, 'S4/N2S4TFA1':15, 'S4/N2S4TFA2':16,
                      'S1/N3S1TFA1':17, 'S1/N3S1TFA2':18, 'S2/N3S2TFA1':19, 'S2/N3S2TFA2':20,
                      'S3/N3S3TFA1':21, 'S3/N3S3TFA2':22, 'S4/N3S4TFA1':23, 'S4/N3S4TFA2':24,
                      'S1/N4S1TFA1':25, 'S1/N4S1TFA2':26, 'S2/N4S2TFA1':27, 'S2/N4S2TFA2':28,
                      'S3/N4S3TFA1':29, 'S3/N4S3TFA2':30, 'S4/N4S4TFA1':31, 'S4/N4S4TFA2':32}

event_dict_N = {'N1':1, 'N2':2, 'N3':3, 'N4':4, 'Smaller Match':5, 'Larger Match':6, 'Left Button':7, 'Right Button':8}

event_dict_S = {'S1':1, 'S2':2, 'S3':3, 'S4':4, 'Smaller Match':5, 'Larger Match':6, 'Left Button':7, 'Right Button':8}

event_dict_TFA = {'TFA1':1, 'TFA2':2, 'Smaller Match':3, 'Larger Match':4, 'Left Button':5, 'Right Button':6}

In [ ]:
def save_multi_image(figs,fname):
    pdf = PdfPages(fname)
    for fig in range(len(figs)):
        pdf.savefig(figs[fig])
    pdf.close()

In [ ]:
# Prepare files
# Rename fif files and make fig folder inside meg folder

def prepare_file(rename_file):
    if rename_file:
        os.chdir(raw_path)
        path = glob.glob("*.fif")

        regex = re.compile(r'\d+')

        for fn in path:
            inx = regex.findall(fn)[3]
            os.rename(fn,inx.lstrip('0')+"_"+raw_fname)
            
    if not os.path.exists(figure_path):
        os.makedirs(figure_path)

In [ ]:
def import_data():
    path = os.path.join(raw_path,"%i_"+raw_fname)
    raws = [read_raw_fif(path % run, verbose='error')
            for run in range(1,n_runs+1)]  # ignore filename warnings
    
    return raws

#--------------------------------------------------#

def import_info():
    def loadmat(filename):
        '''
        this function should be called instead of direct spio.loadmat
        as it cures the problem of not properly recovering python dictionaries
        from mat files. It calls the function check keys to cure all entries
        which are still mat-objects
        '''
        def _check_keys(d):
            '''
            checks if entries in dictionary are mat-objects. If yes
            todict is called to change them to nested dictionaries
            '''
            for key in d:
                if isinstance(d[key], scipy.io.matlab.mio5_params.mat_struct):
                    d[key] = _todict(d[key])
            return d

        def _todict(matobj):
            '''
            A recursive function which constructs from matobjects nested dictionaries
            '''
            d = {}
            for strg in matobj._fieldnames:
                elem = matobj.__dict__[strg]
                if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
                    d[strg] = _todict(elem)
                elif isinstance(elem, np.ndarray):
                    d[strg] = _tolist(elem)
                else:
                    d[strg] = elem
            return d

        def _tolist(ndarray):
            '''
            A recursive function which constructs lists from cellarrays
            (which are loaded as numpy ndarrays), recursing into the elements
            if they contain matobjects.
            '''
            elem_list = []
            for sub_elem in ndarray:
                if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
                    elem_list.append(_todict(sub_elem))
                elif isinstance(sub_elem, np.ndarray):
                    elem_list.append(_tolist(sub_elem))
                else:
                    elem_list.append(sub_elem)
            return elem_list
        data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
        return _check_keys(data)
    
    path = os.path.join(information_path,information_fname)
    subject_info = loadmat(path)
    
    return subject_info

In [ ]:
def concatenate_raw_obj(raws, on_mismatch='raise'):
    
    from mne.io.meas_info import _ensure_infos_match
    
    # Make a deepcopy of raws to keep raws intact
    #https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/
    _raws = copy.deepcopy(raws)
    
    for idx, r in enumerate(_raws[1:], start=1):
        _ensure_infos_match(info1=_raws[0].info, info2=r.info, name=f'raws[{idx}]', on_mismatch=on_mismatch)
        
    _raws[0].append(_raws[1:])
    
    return _raws[0]

In [ ]:
def modify_raw_obj(raws, subject_info):
    for r in range(1,n_runs+1):
        drop_channels = subject_info['meg'][subj]['channels']['drop']['run'+str(r)]
        rename_channels = subject_info['meg'][subj]['channels']['rename']['run'+str(r)]

        if rename_channels != 'none':
            channel_dict = ast.literal_eval(rename_channels)
            for c in channel_dict.items():
                try:
                    raws[r-1].rename_channels(dict([c]))
                except:
                    raws[r-1].drop_channels(list(dict([c]).values()))
                    raws[r-1].rename_channels(dict([c]))
        if drop_channels != 'none':
            raws[r-1].drop_channels(drop_channels)

        if not 'EOG061' in subject_info['meg'][subj]['channels']['drop']['run'+str(r)]:
            raws[r-1].set_channel_types({'EOG061':'eog'})
        if not 'EOG062' in subject_info['meg'][subj]['channels']['drop']['run'+str(r)]:
            raws[r-1].set_channel_types({'EOG062':'eog'})
        if not 'ECG063' in subject_info['meg'][subj]['channels']['drop']['run'+str(r)]:
            raws[r-1].set_channel_types({'ECG063':'ecg'})

In [ ]:
def print_info(raw):
    print(raw.info)
    #print()   # insert a blank line in the output
    #print('Bad channels:', raw.info['bads'])
    #print('Sampling frequency:', raw.info['sfreq'], 'Hz')

    #raw.info['chs']
    #raw.info['dig']

    #print(raw.info['bads'])
    #print(raw.info['ch_names'])

    #info = mne.io.read_info(raw)
    #print(info.keys())
    #print(info['ch_names'])
    #print(info['bads'])

In [ ]:
def vis_plot_raw(raw):
    raw.plot(duration=5, n_channels=20, group_by='position')

In [ ]:
def handle_bad_channels(raw):
    raw.info['bads']

    raw.plot_psd(fmax=50, picks=['meg'])
    raw.plot_psd_topo(fmax=50)

In [ ]:
def process_diode(raw):
    """
    Correct event timing using photodiode, get the individual directions timings and calculate
    the delay due to stimuli presentation

    adapted from cimec_correct_diode
    """

    raw_ = raw.copy()
    
    # get the photodiode data
    diode_data = raw_.get_data(picks=raw_.ch_names.index('MISC008')).squeeze()
    # low pass at 15 hz, this makes the correction robust to diode displacements
    diode_data = mne.filter.filter_data(diode_data, raw_.info['sfreq'], l_freq=None, h_freq=15)
    # initialize samples indices
    samples = np.arange(0, len(diode_data))
    # diode goes from gray to black, then continues black for the trajectories
    # here we invert the values so that the first change is positive
    diode_data = np.abs(diode_data - diode_data.max())
    # subject specific "baseline noise", to account for diodes displacement
    diode_min, diode_max = np.amin(diode_data[10:500]), np.amax(diode_data[10:500])
    # create a strict binary mask: 1 == stimulus present
    mask = diode_data >= diode_max
    # find out when it changes to get the onset
    onsets = np.nonzero(np.diff(mask) > 0)[0]
    # find events from triggers
    events_ = mne.find_events(raw_, stim_channel='STI101', min_duration=.005, shortest_event=1, output='onset')
    # keep only the events coded also using the diode
    # Right/Left button keys pressed
    exclude_events = [128+1,128+2,128+4,128+8]
    start_trg = mne.pick_events(events_, exclude=exclude_events)
    # initialize the output
    start_correct = start_trg.copy()
    trj_list = []
    # initialize the figure
    #fig, ax = plt.subplots(len(start_trg), 1, dpi=150, sharey=True, figsize=(15, 8))
    # then correct each event
    for i_event, start_event in enumerate(start_trg):
        # 1) find trial start
        # get the onset of the trial according to meg trigger
        meg_trg_start = start_event[0] - raw_.first_samp
        # define trial samples
        trl_ind = np.arange(meg_trg_start - 100, meg_trg_start + 0.5 * 1000)
        # now find the onsets that are:
        # - after the start according to the meg trigger
        # - within the tolerance value (in samples)
        start_ind = np.nonzero(np.logical_and(onsets - meg_trg_start >= 0,
                                              (onsets - meg_trg_start) <= 150))[0]
        start_correct[i_event, 0] = onsets[min(start_ind)]
        
        """
        # plot
        ax[i_event].plot(samples[trl_ind], diode_data[trl_ind])
        ax[i_event].vlines(samples[meg_trg_start], diode_data.min(), diode_data.max(), 'r', label='meg trigger')
        ax[i_event].vlines(samples[start_correct[i_event, 0]], diode_data.min(), diode_data.max(), 'g', label='corrected')
        ax[i_event].set_xlim(meg_trg_start - 50, meg_trg_start + 500)
        """
        
    # add back the first sample
    start_correct[:, 0] += raw_.first_samp
    
    """
    # display corrected timings
    delta_t = (start_correct[:, 0] - start_trg[:, 0]) / raw_.info['sfreq']
    logging.getLogger('mne').info(f"Photodiode correction: +{np.mean(delta_t):2.2f} ± {np.std(delta_t):2.2f} s")
    plt.suptitle(f"photodiode correction: +{np.mean(delta_t):2.2f}±{np.std(delta_t):2.2f} s")
    ax[0].legend()
    ax[-1].set_xlabel('Time (samples)')
    plt.setp(ax, xticks=[], yticks=[])
    fig.tight_layout()
    fig.savefig(os.path.join(figure_path,diode), dpi=300)
    """
    
    return start_correct

In [ ]:
def gen_annot_interactive(raw):
    fig = raw.plot(duration=5, n_channels=20, group_by='position')
    fig.canvas.key_press_event('a')

In [ ]:
def gen_annot_eog(raw):
    eog_events = mne.preprocessing.find_eog_events(raw)
    eog_onsets = eog_events[:,0] / raw.info['sfreq'] - 0.1
    eog_durations = [0.3] * len(eog_events)
    eog_descriptions = ['BAD_EOG'] * len(eog_events)
    
    eog_annot = mne.Annotations(eog_onsets,eog_durations,eog_descriptions,orig_time=raw.info['meas_date'])

    raw.set_annotations(eog_annot)

In [ ]:
def gen_annot_ecg(raw):
    ecg_events = mne.preprocessing.find_ecg_events(raw)[0]
    ecg_onsets = ecg_events[:, 0] / raw.info['sfreq'] - 0.05
    ecg_durations = [0.15] * len(ecg_events)
    ecg_descriptions = ['BAD_ECG'] * len(ecg_events)
    
    ecg_annot = mne.Annotations(ecg_onsets,ecg_durations,ecg_descriptions,orig_time=raw.info['meas_date'])

    raw.set_annotations(ecg_annot)

In [ ]:
def gen_annot_muscle(raw):
    _raw = raw.copy()
    _raw.load_data()

    # The threshold is data dependent, check the optimal threshold by plotting muscle_scores.
    muscle_threshold = 5  # z-score
    # Choose one channel type, if there are axial gradiometers and magnetometers
    # Select magnetometers as they are more sensitive to muscle activity
    muscle_annot, muscle_scores = mne.preprocessing.annotate_muscle_zscore(_raw, ch_type="mag", threshold=muscle_threshold, min_length_good=0.2, filter_freq=[110, 140])

    _raw.set_annotations(muscle_annot)

    # Plot muscle z-scores across recording
    fig, ax = plt.subplots()
    ax.plot(_raw.times, muscle_scores)
    ax.axhline(y=muscle_threshold, color='r')
    ax.set(xlabel='time, (s)', ylabel='zscore', title='Muscle activity')
    
    fname = str(r+1)+'_muscle'+tail_fname+'.png'
    fig.savefig(os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')
    
    return _raw

In [ ]:
def filter_power_noise(raw,r):
    """
    def add_arrows(axes):
        # add some arrows at 50 Hz and its harmonics
        for ax in axes:
            freqs = ax.lines[-1].get_xdata()
            psds = ax.lines[-1].get_ydata()
            for freq in freqs:
                idx = np.searchsorted(freqs, freq)
                # get ymax of a small region around the freq. of interest
                y = psds[(idx - 4):(idx + 5)].max()
                ax.arrow(x=freqs[idx], y=y + 18, dx=0, dy=-12, color='red',
                         width=0.1, head_width=3, length_includes_head=True)
    """

    _raw = raw.copy()
    _raw.load_data()

    meg_picks = mne.pick_types(_raw.info, meg=True)
    freqs = powerline_freq
    _raw.notch_filter(freqs=freqs, picks=meg_picks)

    figs = []
    for title, data in zip(['Un', 'Notch '], [raw, _raw]):
        fig = data.plot_psd(fmax=250, average=True)
        fig.subplots_adjust(top=0.85)
        fig.suptitle('{}filtered'.format(title), size='xx-large', weight='bold')
        #add_arrows(fig.axes[:2])
        figs.append(fig)
    
    fname = str(r+1)+'_filter_powerline'+tail_fname+'.pdf'
    save_multi_image(figs,os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')
    
    return _raw

In [ ]:
def filter_band(raw,r):
    _raw = raw.copy()
    _raw.load_data()

    meg_picks = mne.pick_types(_raw.info, meg=True)
    _raw.filter(low_pass_freq, high_pass_freq, fir_design='firwin', picks=meg_picks)

    figs = []
    for title, data in zip(['un', ''], [raw, _raw]):
        fig = data.plot_psd(fmax=250, average=True)
        fig.subplots_adjust(top=0.85)
        fig.suptitle('{}filtered'.format(title), size='xx-large', weight='bold')
        fname = 'filter_band_'+'r'+str(r+1)+'_'+'{}filtered'.format(title)+tail_fname+'.png'
        figs.append(fig)
    
    fname = str(r+1)+'_filter_bandpass'+tail_fname+'.pdf'
    save_multi_image(figs,os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')

    return _raw

In [ ]:
def smooth_savgol_raw(raw):
    #Savitzky-Golay filter
    def savgol(x):
        return scipy.signal.savgol_filter(x,window_length,polyorder)

    _raw = raw.copy()
    _raw.load_data()

    meg_picks = mne.pick_types(_raw.info, meg=True)
    _raw.apply_function(savgol, picks=meg_picks, dtype=None, n_jobs=1, channel_wise=True)

    return _raw

In [ ]:
def smooth_butter_raw(raw):
    _raw = raw.copy()
    _raw.load_data()

    iir_params = dict(order=order, ftype='butter')
    meg_picks = mne.pick_types(_raw.info, meg=True)
    _raw.filter(l_freq=l_freq, h_freq=h_freq, picks=meg_picks, method='iir', iir_params=iir_params, verbose=True)

    return _raw

In [ ]:
def downsample_raw(raw):
    #raw.resample(120,npad="auto") # set sampling frequency to 120Hz

    current_sfreq = raw.info['sfreq']
    desired_sfreq = downsample_freq
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    obtained_sfreq = current_sfreq / decim
    lowpass_freq = obtained_sfreq / 3.

    _raw = raw.copy()
    _raw.load_data()

    _raw.filter(l_freq=None, h_freq=lowpass_freq)

    return _raw

In [ ]:
def fit_ica(raw,r):
    figs = []

    # Filtering to remove slow drifts
    # As filtering is a linear operation, the ICA solution found from the filtered signal can be applied to the unfiltered signal
    _raw = raw.copy()
    _raw.load_data().filter(l_freq=1., h_freq=None)

    # Estimate noise covariance matrix from a continuous segment of raw data.
    # It is typically useful to estimate a noise covariance from empty room data or time intervals before starting the stimulation.
    #noise_cov = mne.compute_raw_covariance(raw_erm, tmin=0, tmax=None)

    # Fitting the ICA solution
    ica = ICA(n_components=30, max_iter=1000, random_state=97, method='picard')
    #ica = ICA(n_components=15, max_iter=1000, random_state=97, method='picard', noise_cov=noise_cov)
    ica.fit(_raw)
    
    figs.append(ica.plot_sources(raw, show_scrollbars=False))
    figs.append(ica.plot_components(picks=slice(0,30,1)))

    ecg_idx = mne.pick_types(raw.info, meg=False, eeg=False, stim=False,
                             eog=False, ecg=True, emg=False, ref_meg=False,
                             exclude='bads')
    
    eog_idx = mne.pick_types(raw.info, meg=False, eeg=False, stim=False,
                             eog=True, ecg=False, emg=False, ref_meg=False,
                             exclude='bads')
    
    if list(ecg_idx)!=[]:
        ecg_evoked = create_ecg_epochs(raw).average()
        ecg_evoked.apply_baseline(baseline=(None, -0.2))
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation', threshold='auto')
        ecgAvailable = True
    else:
        ecgAvailable = False
    
    if eog_idx!=[]:
        eog_evoked = create_eog_epochs(raw).average()
        eog_evoked.apply_baseline(baseline=(None, -0.2))
        eog_indices, eog_scores = ica.find_bads_eog(raw)
        eogAvailable = True
    else:
        eogAvailable = False

    # Save figures temporarily for inspection
    fname=str(r+1)+'_temp_figs.pdf'
    save_multi_image(figs,os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')
        
    # If EOG channels are not available, user should select the components manually
    if not eogAvailable:
        eog_indices = []
        print("EOG was not available, consequently, please select the EOG components manually!")
        n = int(input("Please enter the number of EOG component(s) you want to exclude (RUN "+str(r+1) +"): "))
        for i in range(n):
            eog_indices.append(int(input("Please enter the index of the  EOG component(s) you want to exclude one by one: ")))
            
    # If ECG channel is not available, user should select the components manually
    if not ecgAvailable:
        ecg_indices = []
        print("ECG was not available, consequently, please select the ECG components manually!")
        n = int(input("Please enter the number of ECG component(s) you want to exclude (RUN "+str(r+1) +"): "))
        for i in range(n):
            ecg_indices.append(int(input("Please enter the index of the ECG component(s) you want to exclude one by one: ")))
            
    # Delete temporary figures
    os.remove(os.path.join(figure_path,fname))

    if eogAvailable and eog_indices!=[]:
        ica.exclude = []
        ica.exclude = eog_indices

        # Barplot of ICA component "EOG match" scores
        figs.append(ica.plot_scores(eog_scores))

        # Plot diagnostics
        figs = figs + ica.plot_properties(raw, picks=eog_indices)

        # Plot ICs applied to raw data, with EOG matches highlighted
        figs.append(ica.plot_sources(raw, show_scrollbars=False))

        # Plot ICs applied to the averaged EOG epochs, with EOG matches highlighted
        figs.append(ica.plot_sources(eog_evoked))

        ica.exclude = []
        try:
            eog_index = [eog_indices[0]]
        except:
            eog_index = [eog_indices]
    elif eog_indices==[]:
        eog_index = []
    else:
        try:
            eog_index = [eog_indices[0]]
        except:
            eog_index = [eog_indices]

    if ecgAvailable and ecg_indices!=[]:
        ica.exclude = []
        ica.exclude = ecg_indices

        # Barplot of ICA component "ECG match" scores
        figs.append(ica.plot_scores(ecg_scores))

        # Plot diagnostics
        figs = figs + ica.plot_properties(raw, picks=ecg_indices)

        # Plot ICs applied to raw data, with ECG matches highlighted
        figs.append(ica.plot_sources(raw, show_scrollbars=False))

        # Plot ICs applied to the averaged ECG epochs, with ECG matches highlighted
        figs.append(ica.plot_sources(ecg_evoked))

        ica.exclude = []
        try:
            ecg_index = [ecg_indices[0]]
        except:
            ecg_index = [ecg_indices]
    elif ecg_indices==[]:
        ecg_index = []
    else:
        try:
            ecg_index = [ecg_indices[0]]
        except:
            ecg_index = [ecg_indices]

    # Exclude only the first components
    ica.exclude = eog_index+ecg_index
    print("EOG index "+str(eog_index)+" was removed!")
    print("ECG index "+str(ecg_index)+" was removed!")

    if ica.exclude!=[]:
        _raw = raw.copy()
        _raw.load_data()
        ica.apply(_raw)

        fname=str(r+1)+'_components'+tail_fname+'.pdf'
        save_multi_image(figs,os.path.join(figure_path,fname))
        matplotlib.pyplot.close('all')
    else:
        _raw = raw.copy()
        
        fname=str(r+1)+'_components'+tail_fname+'.pdf'
        save_multi_image(figs,os.path.join(figure_path,fname))
        matplotlib.pyplot.close('all')

    return _raw

In [ ]:
def vis_specral_density_raw(raw):
    raw.plot_psd(fmax=50, picks=['meg'])

In [ ]:
def vis_sensors(raw):
    fig = plt.figure()
    ax2d = fig.add_subplot(121)
    ax3d = fig.add_subplot(122, projection='3d')
    raw.plot_sensors(ch_type='grad', axes=ax2d)
    raw.plot_sensors(ch_type='grad', axes=ax3d, kind='3d')
    #ax3d.view_init(azim=70, elev=15)

In [ ]:
def vis_sensors(raw):
    fig = mne.viz.plot_alignment(raw.info, trans=None, dig=True, eeg=False, show_axes=True,
                                 surfaces=[], meg=['helmet', 'sensors'],
                                 coord_frame='meg')

    mne.viz.set_3d_view(fig, azimuth=50, elevation=90, distance=0.5)

In [ ]:
def make_event(raw):
    # Sample Stimulus: 1-32
    # Smaller Match Stimulus: 1-32 + 32
    # Larger Match Stimulus: 1-32 + 64
    # Button Pressed: 1,2,4,8 + 128
    # Right Hand: Yellow (2), Green (4)
    # Left Hand: Red (1), Blue (8)
    # 4 comparision in each block (2 larger, 2 smaller)
    # 16 comarision in each fif files (8 larger, 8 smaller)
    
    figs = []
    
    _raw = raw.copy()
    events = mne.find_events(_raw, stim_channel='STI101', min_duration=.005, shortest_event=1, output='onset')
    #events = process_diode(raw);

    # Merged events for number
    merged_events_num = events
    merged_events_num = mne.merge_events(merged_events_num, list(range(1,9)), 1)   #N1
    merged_events_num = mne.merge_events(merged_events_num, list(range(9,17)), 2)   #N2
    merged_events_num = mne.merge_events(merged_events_num, list(range(17,25)), 3)   #N3
    merged_events_num = mne.merge_events(merged_events_num, list(range(25,33)), 4)   #N4
    merged_events_num = mne.merge_events(merged_events_num, list(range(33,65)), 5)   #Smaller Match
    merged_events_num = mne.merge_events(merged_events_num, list(range(65,129)), 6)   #Larger Match
    merged_events_num = mne.merge_events(merged_events_num, [129,136], 7)   #Left Button
    merged_events_num = mne.merge_events(merged_events_num, [130,132], 8)   #Right Button

    # Merged events for size
    merged_events_size = events
    merged_events_size = mne.merge_events(merged_events_size, [1,2,9,10,17,18,25,26], 1)   #S1
    merged_events_size = mne.merge_events(merged_events_size, [3,4,11,12,19,20,27,28], 2)   #S2
    merged_events_size = mne.merge_events(merged_events_size, [5,6,13,14,21,22,29,30], 3)   #S3
    merged_events_size = mne.merge_events(merged_events_size, [7,8,15,16,23,24,31,32], 4)   #S4
    merged_events_size = mne.merge_events(merged_events_size, list(range(33,65)), 5)   #Smaller Match
    merged_events_size = mne.merge_events(merged_events_size, list(range(65,129)), 6)   #Larger Match
    merged_events_size = mne.merge_events(merged_events_size, [129,136], 7)   #Left Button
    merged_events_size = mne.merge_events(merged_events_size, [130,132], 8)   #Right Button

    # Merged events for TFA
    merged_events_tfa = events
    merged_events_tfa = mne.merge_events(merged_events_tfa, [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31], 1)   #TFA1
    merged_events_tfa = mne.merge_events(merged_events_tfa, [2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32], 2)   #TFA2
    merged_events_tfa = mne.merge_events(merged_events_tfa, list(range(33,65)), 3)   #Smaller Match
    merged_events_tfa = mne.merge_events(merged_events_tfa, list(range(65,129)), 4)   #Larger Match
    merged_events_tfa = mne.merge_events(merged_events_tfa, [129,136], 5)   #Left Button
    merged_events_tfa = mne.merge_events(merged_events_tfa, [130,132], 6)   #Right Button
    
    fig = plt.figure()
    events_exp = np.delete(events, [0,1], axis=1).T
    unique, counts = np.unique(events_exp, return_counts=True)
    dict(zip(unique, counts))
    keys = dict(zip(unique, counts)).keys()
    values = dict(zip(unique, counts)).values()
    plt.bar(keys, values)
    for key, value in dict(zip(unique, counts)).items():
        plt.text(key-0.65, value+0.05, str(value))
    plt.gcf().set_size_inches(30,15)
    figs.append(fig)
    
    # Number
    figs.append(mne.viz.plot_events(merged_events_num, event_id=event_dict_N, sfreq=raw.info['sfreq'], first_samp=raw.first_samp))

    # Size
    figs.append(mne.viz.plot_events(merged_events_size, event_id=event_dict_S, sfreq=raw.info['sfreq'], first_samp=raw.first_samp))

    # TFA
    figs.append(mne.viz.plot_events(merged_events_tfa, event_id=event_dict_TFA, sfreq=raw.info['sfreq'], first_samp=raw.first_samp))
    
    fname = 'events'+tail_fname+'.pdf'
    save_multi_image(figs,os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')

    return events

In [ ]:
def epoch_data(events,raw):
    
    current_sfreq = raw.info['sfreq']
    desired_sfreq = downsample_freq
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    
    """
    # Reject criteria
    reject_criteria = dict(mag=4000e-15,     # 4000 fT
                           grad=4000e-13)    # 4000 fT/cm

    # Flat criteria
    flat_criteria = dict(mag=1e-15,          # 1 fT
                         grad=1e-13)         # 1 fT/cm

    # Epochs with rejecting criteria and annotation
    epochs_sample_N = mne.Epochs(raw, events, event_id=event_sample_dict_N,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim, 
                                 reject=reject_criteria, flat=flat_criteria,
                                 reject_by_annotation=True,
                                 preload=True)

    # Epochs with rejecting annotation
    epochs_sample_N = mne.Epochs(raw, events, event_id=event_sample_dict_N,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim, 
                                 reject_by_annotation=True,
                                 preload=True)
    """

    # Epochs without rejecting criteria and annotation
    epochs_sample_N = mne.Epochs(raw, events, event_id=event_sample_dict_N,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim)

    epochs_sample_TFA = mne.Epochs(raw, events, event_id=event_sample_dict_TFA,
                                   baseline=baseline, tmin=tmin, tmax=tmax, decim=decim)

    epochs_sample_S = mne.Epochs(raw, events, event_id=event_sample_dict_S,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim)

    #################### Number ####################
    epochs_sample_N1 = epochs_sample_N['N1']
    epochs_sample_N2 = epochs_sample_N['N2']
    epochs_sample_N3 = epochs_sample_N['N3']
    epochs_sample_N4 = epochs_sample_N['N4']

    epochs_sample_N.drop_bad()
    epochs_sample_N1.drop_bad()
    epochs_sample_N2.drop_bad()
    epochs_sample_N3.drop_bad()
    epochs_sample_N4.drop_bad()

    #################### TFA ####################
    epochs_sample_TFA1 = epochs_sample_TFA['TFA1']
    epochs_sample_TFA2 = epochs_sample_TFA['TFA2']

    epochs_sample_TFA.drop_bad()
    epochs_sample_TFA1.drop_bad()
    epochs_sample_TFA2.drop_bad()

    #################### Size ####################
    epochs_sample_S1 = epochs_sample_S['S1']
    epochs_sample_S2 = epochs_sample_S['S2']
    epochs_sample_S3 = epochs_sample_S['S3']
    epochs_sample_S4 = epochs_sample_S['S4']

    epochs_sample_S.drop_bad()
    epochs_sample_S1.drop_bad()
    epochs_sample_S2.drop_bad()
    epochs_sample_S3.drop_bad()
    epochs_sample_S4.drop_bad()
    
    epochs = {
        "N": epochs_sample_N,
        "S": epochs_sample_S,
        "TFA": epochs_sample_TFA,
        "N1": epochs_sample_N1,
        "N2": epochs_sample_N2,
        "N3": epochs_sample_N3,
        "N4": epochs_sample_N4,
        "S1": epochs_sample_S1,
        "S2": epochs_sample_S2,
        "S3": epochs_sample_S3,
        "S4": epochs_sample_S4,
        "TFA1": epochs_sample_TFA1,
        "TFA2": epochs_sample_TFA2}

    return epochs

In [ ]:
def estimate_evoked(epochs):
    evoked_N1 = epochs["N1"].average()
    evoked_N2 = epochs["N2"].average()
    evoked_N3 = epochs["N3"].average()
    evoked_N4 = epochs["N4"].average()

    evoked_TFA1 = epochs["TFA1"].average()
    evoked_TFA2 = epochs["TFA2"].average()

    evoked_S1 = epochs["S1"].average()
    evoked_S2 = epochs["S2"].average()
    evoked_S3 = epochs["S3"].average()
    evoked_S4 = epochs["S4"].average()
    
    evoked = {
        "N1": evoked_N1,
        "N2": evoked_N2,
        "N3": evoked_N3,
        "N4": evoked_N4,
        "S1": evoked_S1,
        "S2": evoked_S2,
        "S3": evoked_S3,
        "S4": evoked_S4,
        "TFA1": evoked_TFA1,
        "TFA2": evoked_TFA2}
    
    return evoked

def estimate_evoked_autoreject(epochs):
    epochs_N1 = epochs['N1S1TFA1', 'N1S1TFA2', 'N1S2TFA1', 'N1S2TFA2', 'N1S3TFA1', 'N1S3TFA2', 'N1S4TFA1', 'N1S4TFA2']
    epochs_N2 = epochs['N2S1TFA1', 'N2S1TFA2', 'N2S2TFA1', 'N2S2TFA2', 'N2S3TFA1', 'N2S3TFA2', 'N2S4TFA1', 'N2S4TFA2']
    epochs_N3 = epochs['N3S1TFA1', 'N3S1TFA2', 'N3S2TFA1', 'N3S2TFA2', 'N3S3TFA1', 'N3S3TFA2', 'N3S4TFA1', 'N3S4TFA2']
    epochs_N4 = epochs['N4S1TFA1', 'N4S1TFA2', 'N4S2TFA1', 'N4S2TFA2', 'N4S3TFA1', 'N4S3TFA2', 'N4S4TFA1', 'N4S4TFA2']
    epochs_S1 = epochs['N1S1TFA1', 'N1S1TFA2', 'N2S1TFA1', 'N2S1TFA2', 'N3S1TFA1', 'N3S1TFA2', 'N4S1TFA1', 'N4S1TFA2']
    epochs_S2 = epochs['N1S2TFA1', 'N1S2TFA2', 'N2S2TFA1', 'N2S2TFA2', 'N3S2TFA1', 'N3S2TFA2', 'N4S2TFA1', 'N4S2TFA2']
    epochs_S3 = epochs['N1S3TFA1', 'N1S3TFA2', 'N2S3TFA1', 'N2S3TFA2', 'N3S3TFA1', 'N3S3TFA2', 'N4S3TFA1', 'N4S3TFA2']
    epochs_S4 = epochs['N1S4TFA1', 'N1S4TFA2', 'N2S4TFA1', 'N2S4TFA2', 'N3S4TFA1', 'N3S4TFA2', 'N4S4TFA1', 'N4S4TFA2']
    epochs_TFA1 = epochs['N1S1TFA1', 'N1S2TFA1', 'N1S3TFA1', 'N1S4TFA1', 'N2S1TFA1', 'N2S2TFA1', 'N2S3TFA1', 'N2S4TFA1',
                         'N3S1TFA1', 'N3S2TFA1', 'N3S3TFA1', 'N3S4TFA1', 'N4S1TFA1', 'N4S2TFA1', 'N4S3TFA1', 'N4S4TFA1']
    epochs_TFA2 = epochs['N1S1TFA2', 'N1S2TFA2', 'N1S3TFA2', 'N1S4TFA2', 'N2S1TFA2', 'N2S2TFA2', 'N2S3TFA2', 'N2S4TFA2',
                         'N3S1TFA2', 'N3S2TFA2', 'N3S3TFA2', 'N3S4TFA2', 'N4S1TFA2', 'N4S2TFA2', 'N4S3TFA2', 'N4S4TFA2']

    evoked_N1 = epochs_N1.average()
    evoked_N2 = epochs_N2.average()
    evoked_N3 = epochs_N3.average()
    evoked_N4 = epochs_N4.average()
    evoked_TFA1 = epochs_TFA1.average()
    evoked_TFA2 = epochs_TFA2.average()
    evoked_S1 = epochs_S1.average()
    evoked_S2 = epochs_S2.average()
    evoked_S3 = epochs_S3.average()
    evoked_S4 = epochs_S4.average()
    
    evoked = {
        "N1": evoked_N1,
        "N2": evoked_N2,
        "N3": evoked_N3,
        "N4": evoked_N4,
        "S1": evoked_S1,
        "S2": evoked_S2,
        "S3": evoked_S3,
        "S4": evoked_S4,
        "TFA1": evoked_TFA1,
        "TFA2": evoked_TFA2}
    
    return evoked

In [ ]:
def vis_plot_epoch(epochs,_str):
    #_str: "N1", "N2", "N3", "N4", "S1", "S2", "S3", "S4", "TFA1", "TFA2"
    epochs[_str].plot()

In [ ]:
def vis_plot_evoked(evoked):
    
    figs = []
    
    figs = figs + mne.viz.plot_compare_evokeds(dict(N1=evoked["N1"], N2=evoked["N2"], N3=evoked["N3"], N4=evoked["N4"]),
                                               picks= 'meg', legend='upper left', show_sensors='upper right')
    
    figs = figs + mne.viz.plot_compare_evokeds(dict(TFA1=evoked["TFA1"], TFA2=evoked["TFA2"]),
                                               picks= 'meg', legend='upper left', show_sensors='upper right')

    figs = figs + mne.viz.plot_compare_evokeds(dict(S1=evoked["S1"], S2=evoked["S2"], S3=evoked["S3"], S4=evoked["S4"]),
                                               picks= 'meg', legend='upper left', show_sensors='upper right')
    
    fname='evoked'+tail_fname+'.pdf'
    save_multi_image(figs,os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')

In [ ]:
def vis_plot_joint_evoked(evoked,_str):
    #_str: "N1", "N2", "N3", "N4", "S1", "S2", "S3", "S4", "TFA1", "TFA2"
    evoked[_str].plot_joint(picks='mag',times=[0, 0.11, 0.17, 0.27, 0.48])
    evoked[_str].plot_joint(picks='grad',times=[0, 0.11, 0.17, 0.27, 0.48])

In [ ]:
def vis_plot_topo_evoked(evoked,_str):
    #_str: "N1", "N2", "N3", "N4", "S1", "S2", "S3", "S4", "TFA1", "TFA2"
    evoked[_str].pick_types(meg='mag').plot_topo(color='r', legend=False)
    evoked[_str].pick_types(meg='grad').plot_topo(color='r', legend=False)

In [ ]:
def vis_map_topo_evoked(evoked,_str):
    evoked[_str].plot_topomap(times=[0, 0.11, 0.17, 0.27, 0.48], ch_type='mag')
    evoked[_str].plot_topomap(times=[0, 0.11, 0.17, 0.27, 0.48], ch_type='grad')

In [ ]:
def autoreject_detect_bad_sensors(raw):
    
    current_sfreq = raw.info['sfreq']
    desired_sfreq = downsample_freq
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    
    _raw = raw.copy()
    _event = mne.find_events(_raw, stim_channel='STI101', min_duration=.005, shortest_event=1, output='onset')

    # Autoreject can repair only one channel type at a time.
    picks_grad = mne.pick_types(_raw.info, meg='grad', eeg=False, stim=False, eog=False, ecg=False, include=[], exclude=[])
    picks_mag = mne.pick_types(_raw.info, meg='mag', eeg=False, stim=False, eog=False, ecg=False, include=[], exclude=[])

    # Remove proj, don't proj while interpolating
    _raw.del_proj()
    _epochs = mne.Epochs(_raw, _event, event_id=event_sample_dict, baseline=baseline, tmin=tmin, tmax=tmax, decim=decim, preload=True)

    ransac = Ransac(verbose='progressbar', picks=picks_grad, n_jobs=1, min_corr=0.5)
    ransac.fit(_epochs)
    print("Gradiometer:")
    print('\n'.join(ransac.bad_chs_))

    ransac = Ransac(verbose='progressbar', picks=picks_mag, n_jobs=1, min_corr=0.5)
    ransac.fit(_epochs)
    print("Magnetometer:")
    print('\n'.join(ransac.bad_chs_))

In [ ]:
def export_mat(epochs):
    _epochs = copy.deepcopy(epochs)
    _epochs.load_data().pick_types(meg=True, eeg=False, stim=False, eog=False, ecg=False)

    _epochs.drop_bad()
    _epochs.equalize_event_counts(event_sample_list)

    #_epochs.save('mne_python-epo.fif')
    #mne.write_events('mne_python-eve.fif', _epochs.events)

    unique, counts = np.unique(_epochs.events[:,2], return_counts=True)
    dict(zip(unique, counts))
    keys = dict(zip(unique, counts)).keys()
    values = dict(zip(unique, counts)).values()
    print(str(len(unique))+" epoches have been extracted!")
    for key, value in dict(zip(unique, counts)).items():
        print("There are "+str(value)+" events in the epoch "+str(key))

    data = _epochs.get_data()

    # Reduce dimension using PCA
    #pca = UnsupervisedSpatialFilter(PCA(30), average=True)
    #data = pca.fit_transform(data)

    stim_index = np.zeros(len(_epochs.events), dtype=int)
    stim_label = list()
    for i in range(len(_epochs.events)):
        stim_index[i] = _epochs.events[i][2]
        stim_label.append(list(event_sample_dict.keys())[list(event_sample_dict.values()).index(_epochs.events[i][2])])

    epochs_dict = {
        # n_samples * n_channels * n_time
        "trial": data,
        "index": stim_index,
        "label": stim_label,
        "time": np.linspace(1000*tmin,1000*tmax,data.shape[2]),
        "channel_name": _epochs.ch_names,
        "channel_type": _epochs.get_channel_types(picks='meg')}

    fname = epoched_fname
    scipy.io.savemat(os.path.join(epoched_path,fname),epochs_dict)


In [ ]:
########## Import Raw Data ##########
prepare_file(rename_file=True)
raws = import_data()

In [ ]:
########## Inspect Raw Data ##########
run = 1
handle_bad_channels(raws[run-1])
vis_plot_raw(raws[run-1])

In [ ]:
%%capture cap --no-stderr

########## Import Data and Information ########## 
prepare_file(rename_file=False)
raws = import_data()
subject_info = import_info()
modify_raw_obj(raws,subject_info)

for r in range(n_runs):
    ########## Print Information ##########
    print("************************* Data Information - run: "+str(r+1)+" *************************")
    print_info(raws[r])
    
    ########## Filter Data ########## 
    print("************************* Power-line noise filter - run: "+str(r+1)+" *************************")
    raws[r] = filter_power_noise(raws[r],r)

    print("************************* Band-pass filter - run: "+str(r+1)+" *************************")
    raws[r] = filter_band(raws[r],r)

    ########## Run ICA ########## 
    if "ica" in tail_fname:
        print("************************* Run ICA - run: "+str(r+1)+" *************************")
        raws[r] = fit_ica(raws[r],r)

    ########## Annotate Muscle ########## 
    print("************************* Annotate muscle activity - run: "+str(r+1)+" *************************")
    raws[r] = gen_annot_muscle(raws[r])

    ########## Smooth Data ########## 
    print("************************* Smooth data - run: "+str(r+1)+" *************************")
    raws[r] = smooth_savgol_raw(raws[r])
    #raws[r] = smooth_butter_raw(raws[r])

    ########## Downsample Data ########## 
    print("************************* Downsample data - run: "+str(r+1)+" *************************")
    raws[r] = downsample_raw(raws[r])
    
    ########## Pick MEG ##########
    raws[r].pick_types(meg=True, eeg=False, stim=True, misc=True, eog=False, ecg=False, include=[], exclude=[])

########## Concatenate Data ##########
print("************************* Concatenating raws *************************")
raw = concatenate_raw_obj(raws,on_mismatch='warn')

########## Create Events, Epochs, and Evoked Data then Save Epochs ##########
events = make_event(raw)
epochs = epoch_data(events,raw)
evoked = estimate_evoked(epochs)
vis_plot_evoked(evoked)
export_mat(epochs["N"])

In [ ]:
########## Write Log ##########
fname = log_fname
with open(os.path.join(log_path,fname), 'w') as f:
    f.write(cap.stdout)